In [ ]:
import time,serial,random
import numpy as np
import scipy,pybrain3

def servoin(a=90,b=90,c=90,d=90,e=90,f=90,delay=1):
    '''waits end of previous act. and send angles to arduino returns acc-array     
    '''
    while ser.read()!=b'>':
        pass
    acc = ser.readline().split(b'\t')
    A=np.array([[float(acc[0])],[float(acc[1])],[float(acc[2])]])
    print(*A)
    if fall_check(*A)==False:
        data='in'
        for i in a,b,c,d,e,f,delay:
            try: 
                data+=str(hex(i))[2]+str(hex(i))[3]
            except IndexError:
                data+='0'+str(hex(i))[2]
        ser.write(data.encode())
        print(data)
    else:
        print('I have fallen')
        1/0

    
def fall_check(accelx,accely,accelz):
    '''gets measurement of accelerometer
        checks if Robo has fallen
        returns False if Robo - NOT fallen
        returns True if Robo - has fallen'''
    if (abs(accelx)>5 or abs(accelz-2)>5) and accely<7:
        return True
    else: return False
    
def interpreter(st,old):
    '''gets input string and last value of servoin
        returns tuple of updated values servo
        if str='start' to initials
        if str='a=12 c=13' returns (12,90,13,90,90,90,1)
        if str='(12,90,13,90,90,90,1)' returns the same
        if str='(12,90,13,90)' returns the same
        if str='a=12 c=13 then e=50 then f=40 then start' compile the algorythm
        if str='a=5 c=3 then while a=6 b=2 and a=4 b=4' compile
        if str='a=5 c=3 then wait 3 then b=5' compile
    '''
    if st=='start':
        old = [90,90,90,90,90,90,1]
        servoin(*old)
        return old
    
    if st=='jackson':
        old = interpreter('start for 8 b=120 d=120 and a=120 c=120 and wait 1 and a=60 c=60 and b=60 d=60')
        
    if st=='walk':
        old = interpreter('start then for 8 e=80 g=3 and f=80 b=70 a=70 and a=50 and d=70 f=90 e=90 c=70 and e=110 f=110 b=100 a=90 d=90 and c=100 e=120 b=120 g=1 and b=130 d=110 a=120 c=120 and c=140 e=90 f=90 g=3 and e=80 f=80 d=80 b=90 a=60 c=80',old)
          
    if st[0:4]=='wait':
        time.sleep(int(st[5]))
        old=interpreter(st[7:],old)
    
    if st[0:5]=='while':
        while True:
            for j in st[6:].split(' and '):
                old=interpreter(j,old)
    
    if st[0:3]=='for':
        for l in range(int(st[4])): 
            for j in st[6:].split(' and '):
                old=interpreter(j,old)
        return old
    
    try: #12,13,15
        q = list(int(i) for  i in st.split(','))
        for j in range(len(q),7): q.append(old[j])
        old = q
        servoin(*old)
        return old
    except ValueError:
        try: #a=12 b=13
            q = old
            tmp = list(st.split())
            for i in tmp: 
                q[ord(str(i).split('=')[0])-97]=int(str(i).split('=')[1])
            old = q
            servoin(*old)
            return old
        except IndexError: #a=12 b=13 then b=15
            for j in st.split(' then '):
                old=interpreter(str(j),old)
            return old

        
def serial_begin(port):
    ser = serial.Serial(port, 19200, bytesize=8, parity='N', stopbits=1, timeout=2)
    time.sleep(2)
    print("connected to: "+ ser.portstr)
    return [ser,[90,90,90,90,90,90,1]]

if __name__ == "__main__":
    port = 'COM9'
    [ser,old_q]=serial_begin(port)
    while True:
        try:
            inp=input()
            old_q=interpreter(inp,old_q)
        except ZeroDivisionError:
            ser.close()
            [ser,old_q]=serial_begin(port)
    ser.close()


connected to: COM9
walk
[2.28] [9.22] [2.82]
in5a5a5a5a5a5a01
[0.16] [10.04] [2.28]
in5a5a5a5a505a03
[1.69] [10.87] [2.55]
in46465a5a505003
[-0.82] [7.26] [2.31]
in32465a5a505003
[0.12] [10.12] [-0.43]
in324646465a5a03
[7.1] [6.16] [-0.51]
I have fallen
connected to: COM9
walk
[8.] [9.18] [-3.92]
in5a5a5a5a5a5a01
[-0.04] [9.96] [2.24]
in5a5a5a5a505a03
[1.22] [10.16] [2.51]
in46465a5a505003
[-1.37] [7.34] [3.49]
in32465a5a505003
[0.78] [8.87] [-1.26]
in324646465a5a03
[8.71] [2.86] [-2.59]
I have fallen
connected to: COM9
a=50
[-3.18] [14.83] [2.24]
in325a5a5a5a5a01
start
[2.47] [11.18] [1.29]
in5a5a5a5a5a5a01
walk
[2.08] [6.43] [5.53]
in5a5a5a5a5a5a01
[-0.39] [9.96] [2.24]
in5a5a5a5a505a03
[1.37] [10.36] [2.35]
in46465a5a505003
[-0.31] [6.24] [4.75]
in32465a5a505003
[11.14] [3.45] [-6.47]
I have fallen
connected to: COM9
walk
[8.36] [5.69] [-4.55]
I have fallen
connected to: COM9
walk
[1.26] [9.41] [2.43]
in5a5a5a5a5a5a01
[0.16] [10.16] [2.28]
in5a5a5a5a505a03
[2.12] [10.79] [2.47]
in46

In [ ]:
ser.close()